In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from core import EV_Data, Wrap_PyBamm_Output
%load_ext autoreload
%autoreload 2
# Load the .mat file (assuming 'data_path' is defined and points to your .mat file)
%reload_ext autoreload

In [ ]:
import glob
import numpy as np
import pandas as pd
reduce_by = 1
version = "W04"
base_data_path="/home/ccanodom/data/EV_battery/" + version + "/" + version + "_" 
values = ["C01","C03","C07"]
def reduce_by(reduce_by):
    # Assuming df is your DataFrame
    # Step 1: Create a group identifier for each set of 20 rows
    group_identifier = np.arange(len(df)) // reduce_by

    # Step 2 & 3: Group by this identifier and calculate the mean for each group
    df_mean = df.groupby(group_identifier).mean()

    # If your DataFrame has a MultiIndex and you want to keep the non-numeric levels,
    # you may need to reset the index first, or handle them separately depending on your specific case.

    df_mean.reset_index(drop=True, inplace=True)
    df_mean["Step"] = (np.abs(df_mean["Step"]))
    return df_mean


list_df = []
for path in values:
    data_path = base_data_path + path + ".mat"
    print(f"Start with data in path {data_path}")
    df = EV_Data.get_data_EV(data_path)
    df_mean = reduce_by(30)
    
    list_df.append(df_mean)
df = pd.concat(list_df)
df = EV_Data.include_cycle_number(df)
df = EV_Data.map_index(df)

In [ ]:
import matplotlib.pyplot as plt
import bokeh.io
%load_ext autoreload
%autoreload 2
# this is here only for completeness to clarify where
# the methods are nested (you probably already imported this earlier)


bokeh.io.reset_output()
bokeh.io.output_notebook()
    

In [ ]:
Wrap_PyBamm_Output.plot_variable_vs_time(df, "V",6)

In [ ]:
Wrap_PyBamm_Output.plot_variable_vs_time(df, "C",3)

In [ ]:
Wrap_PyBamm_Output.plot_variable_vs_capacity(df, step_index=5, variable_name='V', capacity_type='discharge')

In [ ]:
from core import Wrap_PyBamm_Output
Wrap_PyBamm_Output.plot_variable_vs_capacity(df, step_index=6, variable_name='V', capacity_type='discharge')


In [ ]:
Wrap_PyBamm_Output.plot_variable_vs_capacity(df, step_index=5, variable_name='V', capacity_type='discharge')

In [ ]:
#plot_variable_vs_capacity(df, step_index=9, variable_name='Voltage', capacity_type='charge')
from core import Wrap_PyBamm_Output
Wrap_PyBamm_Output.plot_variable_vs_capacity(df, step_index=2, variable_name='V', capacity_type='charge')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Assuming 'df' is your DataFrame and has been properly filtered for the specific conditions
# and 'Cycle' and 'C_rate' columns exist as described.

# Filter the DataFrame for the specific step, if needed
filtered_df = df[df["Step_Index"] == 14]

# Group the filtered DataFrame by 'Cycle'
grouped_by_cycle = filtered_df.groupby('Cycle_Number')

# Define the colormap
cmap = cm.get_cmap('viridis')

# Get the total number of cycles to normalize the cycle number to [0, 1]
num_cycles = len(grouped_by_cycle)
norm = mcolors.Normalize(vmin=0, vmax=num_cycles)

# Create a plot
# Create a plot and specify axes
fig, ax = plt.subplots(figsize=(12, 8))
discharge_capacity_diff = np.zeros(num_cycles)
cycles = np.zeros(num_cycles)

# Iterate over each cycle to calculate the difference in 'C_rate' from beginning to end
for i, (cycle_number, group) in enumerate(grouped_by_cycle):
    D_cap = group['Time'].to_numpy()
    c_max = D_cap.max() # Assuming the last value represents the end of the cycle
    c_min = D_cap.min()   # Assuming the first value represents the start of the cycle
    
    V = group['Voltage'].to_numpy()
    v_max = V.max() # Assuming the last value represents the end of the cycle
    v_min = V.min()    # Assuming the first value represents the start of the cycle
    
    discharge_capacity_diff[i] = (np.diff(V) / np.diff(D_cap)).mean() #(v_max - v_min) / ((c_max - c_min) / 3600 )# Calculate the difference
    discharge_capacity_diff[i] = (v_max - v_min) / ((c_max - c_min) / 3600 )# Calculate the difference
    
    cycles[i] = cycle_number

# Plot each cycle's discharge capacity difference
# Note: The color normalization line is unnecessary for scatter plots where each point is the same color,
# but it's useful if you want to map the cycle numbers to colors on a gradient.
colors = cmap(norm(cycles))
#plt.scatter(cycles, discharge_capacity_diff, c=colors)
sc = ax.scatter(cycles, discharge_capacity_diff, c=colors)

plt.title('Discharge Capacity Difference by Cycle')
plt.xlabel('Cycle Number')
plt.ylabel('Discharge Capacity Difference')
# Create a colorbar with the correct axes
cbar = plt.colorbar(sc, ax=ax, label='Normalized Cycle Number')
cbar.set_label('Cycle Number', rotation=270, labelpad=15)

plt.show()


In [ ]:
Wrap_PyBamm_Output.plot_av_at_by_cycle(df,3, Wrap_PyBamm_Output.TypeAgregation.deltaC)

In [ ]:
# Create a plot
plt.figure(figsize=(10, 6))

for name, group in grouped:
    plt.plot(group['Discharge_Capacity'], group['Voltage'], label=f'Cycle {name}')

plt.title('Voltage vs. Discharge Capacity for USSD Discharging by Cycle')
plt.xlabel('Discharge Capacity (Ah)')
plt.ylabel('Voltage (V)')
plt.legend()  # Display a legend to identify each cycle
plt.show()

In [ ]:
# Initialize a cycle number array with zeros, same length as step_index
cycle_numbers = np.zeros_like(step_index)

# Start with cycle number 1
current_cycle = 1

# Loop through the step index array to identify changes indicating a new cycle
for i in range(1, len(step_index)):
    # If the step index decreases, it indicates a new cycle has started
    if step_index[i] < step_index[i-1]:
        current_cycle += 1
    cycle_numbers[i] = current_cycle

# Checking the first few cycle numbers to verify our approach
cycle_numbers[:10], np.unique(cycle_numbers)

In [ ]:
Wrap_PyBamm_Output.plot_variable_vs_time_by_cycle(df, "V", 1)

In [ ]:
Wrap_PyBamm_Output.plot_voltage_current_by_cycle(df, 43)